In [1]:
import sys
import cmdstanpy
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import pandas as pd
import json

sys.path.insert(0, '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF')
sys.path.insert(0, '/home/garren/Documents/MEng/Code/Latest_results/HPC Files')

from Post_procs import PostProcess
from All_code import subsets
import os 

from IPython.display import clear_output
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
functional_groups = np.array(['all'])
variance_known = True
inf_type = 'MAP'

In [3]:
## Obtain variables of interest for data without clusters 
post_obj = PostProcess(functional_groups=functional_groups, 
                       include_clusters=False, 
                       variance_known=variance_known, 
                       inf_type=inf_type)

data_dict_c_0 = post_obj.get_MC()
rec_dict_c_0 = post_obj.get_reconstructed_errors()
err_metrics_c_0 = post_obj.compute_error_metrics(data_dict=data_dict_c_0, is_testing=True)
rec_err_metrics_c_0 = post_obj.compute_error_metrics(data_dict=rec_dict_c_0, is_testing=False)
A_c_0 = post_obj.get_pred_param_matrix()

In [4]:
## Obtain variables of interest for data with clusters
post_obj = PostProcess(functional_groups=functional_groups, 
                       include_clusters=True, 
                       variance_known=variance_known, 
                       inf_type=inf_type)

data_dict_c_1 = post_obj.get_MC()
rec_dict_c_1 = post_obj.get_reconstructed_errors()
err_metrics_c_1 = post_obj.compute_error_metrics(data_dict=data_dict_c_1, is_testing=True)
rec_err_metrics_c_1 = post_obj.compute_error_metrics(data_dict=rec_dict_c_1, is_testing=False)
A_c_1 = post_obj.get_pred_param_matrix()

In [ ]:
## Plot the training error distributions for the two cases
bins = 50
er_MC_c_0 = np.abs(data_dict_c_0['y_MC'] - data_dict_c_0['y_exp'])
er_MC_c_1 = np.abs(data_dict_c_1['y_MC'] - data_dict_c_1['y_exp'])
er_UNIFAC = np.abs(data_dict_c_0['y_UNIFAC'] - data_dict_c_0['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC_c_0[er_MC_c_0 > bin_edges[-1]] = bin_edges[-1]
er_MC_c_1[er_MC_c_1 > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Error_plots/Testing'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC_c_0, bins=bin_edges, color='b', alpha=0.5, label='MC - No clusters')
ax[0].hist(er_MC_c_1, bins=bin_edges, color='r', alpha=0.5, label='MC - With clusters')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(data_dict_c_0['y_MC'], data_dict_c_0['y_exp'], '.b', label='MC - No clusters', alpha=0.5)
ax[1].plot(data_dict_c_1['y_MC'], data_dict_c_1['y_exp'], '.r', label='MC - With clusters', alpha=0.5)
ax[1].plot(data_dict_c_0['y_UNIFAC'], data_dict_c_0['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(data_dict_c_0['y_exp']), max(data_dict_c_0['y_MC']), max(data_dict_c_1['y_MC']), max(data_dict_c_0['y_UNIFAC']))
min_lim = min(min(data_dict_c_0['y_exp']), min(data_dict_c_0['y_MC']), min(data_dict_c_1['y_MC']), min(data_dict_c_0['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
## Plot the training error distributions for the two cases
bins = 50
er_MC_c_0 = np.abs(rec_dict_c_0['y_MC'] - rec_dict_c_0['y_exp'])
er_MC_c_1 = np.abs(rec_dict_c_1['y_MC'] - rec_dict_c_1['y_exp'])
er_UNIFAC = np.abs(rec_dict_c_0['y_UNIFAC'] - rec_dict_c_0['y_exp'])
min_val = 0
max_val = 1000

bin_edges = np.linspace(min_val, max_val, bins + 1)
bin_edges = np.append(bin_edges, max_val + bin_edges[1] - bin_edges[0])

er_MC_c_0[er_MC_c_0 > bin_edges[-1]] = bin_edges[-1]
er_MC_c_1[er_MC_c_1 > bin_edges[-1]] = bin_edges[-1]
er_UNIFAC[er_UNIFAC > bin_edges[-1]] = bin_edges[-1]

fig, ax = plt.subplots(2, figsize=(10, 10))

path = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Error_plots/Training'
if not os.path.exists(path):
    os.makedirs(path)

# Plot histograms with the same bin edges
ax[0].hist(er_MC_c_0, bins=bin_edges, color='b', alpha=0.5, label='MC - No clusters')
ax[0].hist(er_MC_c_1, bins=bin_edges, color='r', alpha=0.5, label='MC - With clusters')
ax[0].hist(er_UNIFAC, bins=bin_edges, color='g', alpha=0.5, label='UNIFAC')
ax[0].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[0].set_ylabel('Frequency')
ax[0].set_xlabel('Absolute error')
ax[0].set_xticks(ticks=[0, 200, 400, 600, 800, 1000], labels=['0', '200', '400', '600', '800', '1000+'])

ax[1].plot(rec_dict_c_0['y_MC'], rec_dict_c_0['y_exp'], '.b', label='MC - No clusters', alpha=0.5)
ax[1].plot(rec_dict_c_1['y_MC'], rec_dict_c_1['y_exp'], '.r', label='MC - With clusters', alpha=0.5)
ax[1].plot(rec_dict_c_0['y_UNIFAC'], rec_dict_c_0['y_exp'], '.g', label='UNIFAC', alpha=0.5)
ax[1].legend(loc='upper right', bbox_to_anchor=(1.3, 1))
ax[1].set_ylabel('Experimental')
ax[1].set_xlabel('Predicted')
max_lim = max(max(rec_dict_c_0['y_exp']), max(rec_dict_c_0['y_MC']), max(rec_dict_c_1['y_MC']), max(rec_dict_c_0['y_UNIFAC']))
min_lim = min(min(rec_dict_c_0['y_exp']), min(rec_dict_c_0['y_MC']), min(rec_dict_c_1['y_MC']), min(rec_dict_c_0['y_UNIFAC']))
ax[1].plot([min_lim, max_lim], [min_lim, max_lim], 'k--')
plt.tight_layout()

fig.savefig(f'{path}/Err_dist.png', dpi=300)

In [ ]:
## Plot the 2D plots for the experimental data - testing (some plots should be here)
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    path += f'_{func}'
path += '/2D_plots/Testing'

excel_UNI_sheet = 'Testing_Plots'
_, Idx_known = post_obj.get_excess_enthalpy()
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(data_dict_c_0['c1'].astype(str), ' + '), data_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(data_dict_c_0['c1'], ' + '), data_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    y_idx = exp_mix == unique_mix[j]
    UNIFAC_idx = UNIFAC_mix == unique_mix[j]
    yy = data_dict_c_0['y_exp'][y_idx]
    yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    x_y = data_dict_c_0['x'][y_idx]
    T_y = data_dict_c_0['T'][y_idx]
    c1 = data_dict_c_0['c1'][y_idx][0]
    c2 = data_dict_c_0['c2'][y_idx][0]

    x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

    p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

    p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

    T_uniq = np.unique(T_UNIFAC)
    for i in range(len(T_uniq)):
        if not os.path.exists(f'{path}/{j}_{i}.png'):
            print(f'{j+1} out of {len(unique_mix)} mixtures')
            print(f'{i+1} out of {len(T_uniq)} temperatures')
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT

            fig, ax = plt.subplots()
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], '-g', label='UNIFAC')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], '-b', label='Mean MC - No clusters')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], '-r', label='Mean MC - With clusters')
            ax.plot(x_y[T_y_idx], yy[T_y_idx], '.k', label='Experimental Data')
            ax.set_xlabel('Composition of Compound 1 [mol/mol]')
            ax.set_ylabel('Excess Enthalpy [J/mol]')
            ax.set_title(f'(1) {c1} + (2) {c2} at {T_uniq[i]:.2f} K')
            ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            fig.savefig(f'{path}/{j}_{i}.png', dpi=300)
            plt.close(fig)
            clear_output(wait=False)

In [ ]:
## Plot the 3D plots for the experimental data - testing
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    path += f'_{func}'
path += '/3D_plots/Testing'

excel_UNI_sheet = 'Testing_Plots'
_, Idx_known = post_obj.get_excess_enthalpy()
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(data_dict_c_0['c1'].astype(str), ' + '), data_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(data_dict_c_0['c1'], ' + '), data_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    if not os.path.exists(f'{path}/{j}.png'):
        print(f'{j+1} out of {len(unique_mix)} mixtures')
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        y_idx = exp_mix == unique_mix[j]
        UNIFAC_idx = UNIFAC_mix == unique_mix[j]
        yy = data_dict_c_0['y_exp'][y_idx]
        yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        x_y = data_dict_c_0['x'][y_idx]
        T_y = data_dict_c_0['T'][y_idx]
        c1 = data_dict_c_0['c1'][y_idx][0]
        c2 = data_dict_c_0['c2'][y_idx][0]

        x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

        p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

        p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

        T_uniq = np.unique(T_UNIFAC)
        for i in range(len(T_uniq)):
            # Plot median prediction
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT
            if j == 0:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b', label='Mean MC - No clusters')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r', label='Mean MC - With clusters')
            else:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], c='g', label='UNIFAC')

        # Scatter plot for experimental data
        ax.scatter(x_y, T_y, yy, c='k', marker='.', s=100, label='Experimental Data')

        # Custom legend
        custom_lines = [
            Line2D([0], [0], color='k', marker='.', linestyle='None', markersize=10),  # Experimental Data
            Line2D([0], [0], color='g', lw=4),  # UNIFAC
            Line2D([0], [0], color='b', lw=4), # Mean MC
            Line2D([0], [0], color='r', lw=4)
        ]

        ax.legend(custom_lines, ['Experimental', 'UNIFAC', 'MC - No cluster', 'MC - With clusters'], loc='upper left', bbox_to_anchor=(1.03, 1))

        ax.set_xlabel('Composition of component 1 [mol//mol]', fontsize=14)
        ax.set_ylabel('Temperature [K]', fontsize=14)
        ax.set_zlabel('Excess Enthalpy [J/mol]', fontsize=14, labelpad=10)
        ax.set_title(f'(1) {c1} + (2) {c2}', fontsize=20)
        plt.tight_layout()  # Adjust layout to make room for the legend

        # Disable offset for all axes
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        ax.get_yaxis().get_major_formatter().set_useOffset(False)
        ax.get_zaxis().get_major_formatter().set_useOffset(False)

        plt.savefig(f'{path}/{j}.png', dpi=300)

        plt.close(fig)
        clear_output(wait=False)

In [ ]:
## Plot the 2D plots for the experimental data - training (some plots should be here)
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    path += f'_{func}'
path += '/2D_plots/Training'
excel_UNI_sheet = 'Training_Plots'
Idx_known = np.array(json.load(open(post_obj.data_file, 'r'))['Idx_known']) - 1
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')
    
all_mix = np.char.add(np.char.add(rec_dict_c_0['c1'].astype(str), ' + '), rec_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(rec_dict_c_0['c1'], ' + '), rec_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    y_idx = exp_mix == unique_mix[j]
    UNIFAC_idx = UNIFAC_mix == unique_mix[j]
    yy = rec_dict_c_0['y_exp'][y_idx]
    yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    x_y = rec_dict_c_0['x'][y_idx]
    T_y = rec_dict_c_0['T'][y_idx]
    c1 = rec_dict_c_0['c1'][y_idx][0]
    c2 = rec_dict_c_0['c2'][y_idx][0]

    x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
    T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

    p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

    p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
    yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

    T_uniq = np.unique(T_UNIFAC)
    for i in range(len(T_uniq)):
        if not os.path.exists(f'{path}/{j}_{i}.png'):
            print(f'{j+1} out of {len(unique_mix)} mixtures')
            print(f'{i+1} out of {len(T_uniq)} temperatures')
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT

            fig, ax = plt.subplots()
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], '-g', label='UNIFAC')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], '-b', label='Mean MC - No clusters')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], '-r', label='Mean MC - With clusters')
            ax.plot(x_y[T_y_idx], yy[T_y_idx], '.k', label='Experimental Data')
            ax.set_xlabel('Composition of Compound 1 [mol/mol]')
            ax.set_ylabel('Excess Enthalpy [J/mol]')
            ax.set_title(f'(1) {c1} + (2) {c2} at {T_uniq[i]:.2f} K')
            ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
            plt.tight_layout()

            fig.savefig(f'{path}/{j}_{i}.png', dpi=300)
            plt.close(fig)
            clear_output(wait=False)

            try:
                del fig
            except:
                continue
            del ax

In [ ]:
## Plot the 3D plots for the experimental data - training
excel_UNI_plot = '/home/garren/Documents/MEng/Code/Latest_results/HPC Files/UNIFAC_Plots.xlsx'
path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    path += f'_{func}'
path += '/3D_plots/Training'

excel_UNI_sheet = 'Training_Plots'
Idx_known = np.array(json.load(open(post_obj.data_file, 'r'))['Idx_known']) - 1
try:
    os.makedirs(path)
except:
    print(f'Directory {path} already exists')

all_mix = np.char.add(np.char.add(rec_dict_c_0['c1'].astype(str), ' + '), rec_dict_c_0['c2'].astype(str))
unique_mix, idx = np.unique(all_mix, return_index=True)
unique_mix = unique_mix[np.argsort(idx)]

df_UNIFAC = pd.read_excel(excel_UNI_plot, sheet_name=excel_UNI_sheet)

exp_mix = np.char.add(np.char.add(rec_dict_c_0['c1'], ' + '), rec_dict_c_0['c2'])
UNIFAC_mix = np.char.add(np.char.add(df_UNIFAC['Component 1'].to_numpy().astype(str), ' + '), df_UNIFAC['Component 2'].to_numpy().astype(str))

for j in range(len(unique_mix)):
    if not os.path.exists(f'{path}/{j}.png'):
        print(f'{j+1} out of {len(unique_mix)} mixtures')
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection='3d')
        y_idx = exp_mix == unique_mix[j]
        UNIFAC_idx = UNIFAC_mix == unique_mix[j]
        yy = rec_dict_c_0['y_exp'][y_idx]
        yy_UNIFAC = df_UNIFAC['UNIFAC_DMD [J/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        x_y = rec_dict_c_0['x'][y_idx]
        T_y = rec_dict_c_0['T'][y_idx]
        c1 = rec_dict_c_0['c1'][y_idx][0]
        c2 = rec_dict_c_0['c2'][y_idx][0]

        x_UNIFAC = df_UNIFAC['Composition component 1 [mol/mol]'].to_numpy().astype(float)[UNIFAC_idx]
        T_UNIFAC = df_UNIFAC['Temperature [K]'].to_numpy().astype(float)[UNIFAC_idx]

        p12_c_0 = A_c_0[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_0 = A_c_0[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_0 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_0, p21=p21_c_0)

        p12_c_1 = A_c_1[:, Idx_known[j,0], Idx_known[j,1]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        p21_c_1 = A_c_1[:, Idx_known[j,1], Idx_known[j,0]] * np.array(json.load(open(post_obj.data_file, 'r'))['scaling'])
        yy_MC_mean_c_1 = post_obj.excess_enthalpy_predictions(x=x_UNIFAC, T=T_UNIFAC, p12=p12_c_1, p21=p21_c_1)

        T_uniq = np.unique(T_UNIFAC)
        for i in range(len(T_uniq)):
            # Plot median prediction
            TT = T_uniq[i]
            T_y_idx = np.abs(T_y - TT) <= 0.5
            T_UNIFAC_idx = T_UNIFAC == TT
            if j == 0:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b', label='Mean MC - No clusters')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r', label='Mean MC - With clusters')
            else:
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_0[T_UNIFAC_idx], c='b')
                ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_MC_mean_c_1[T_UNIFAC_idx], c='r')
            ax.plot(x_UNIFAC[T_UNIFAC_idx], T_UNIFAC[T_UNIFAC_idx], yy_UNIFAC[T_UNIFAC_idx], c='g', label='UNIFAC')

        # Scatter plot for experimental data
        ax.scatter(x_y, T_y, yy, c='k', marker='.', s=100, label='Experimental Data')

        # Custom legend
        custom_lines = [
            Line2D([0], [0], color='k', marker='.', linestyle='None', markersize=10),  # Experimental Data
            Line2D([0], [0], color='g', lw=4),  # UNIFAC
            Line2D([0], [0], color='b', lw=4), # Mean MC
            Line2D([0], [0], color='r', lw=4)
        ]

        ax.legend(custom_lines, ['Experimental', 'UNIFAC', 'MC - No cluster', 'MC - With clusters'], loc='upper left', bbox_to_anchor=(1.03, 1))

        ax.set_xlabel('Composition of component 1 [mol//mol]', fontsize=14)
        ax.set_ylabel('Temperature [K]', fontsize=14)
        ax.set_zlabel('Excess Enthalpy [J/mol]', fontsize=14, labelpad=10)
        ax.set_title(f'(1) {c1} + (2) {c2}', fontsize=20)
        plt.tight_layout()  # Adjust layout to make room for the legend

        # Disable offset for all axes
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        ax.get_yaxis().get_major_formatter().set_useOffset(False)
        ax.get_zaxis().get_major_formatter().set_useOffset(False)

        plt.savefig(f'{path}/{j}.png', dpi=300)

        plt.close(fig)
        clear_output(wait=False)

        try: 
            del fig
        except:
            continue
        del ax

In [ ]:
## Save error metrics to excel File
# get testing error metrics
all_err_metrics = err_metrics_c_0.copy()
all_cols = all_err_metrics.columns.to_list()
apppend_cols = []
for i in range(len(all_cols)):
    if all_cols[i][0] == 'MC':
        t = all_cols[i][1]
        all_cols[i] = ('MC - No Cluster', t)
        apppend_cols += [('MC - With Cluster', t)]
all_err_metrics.columns = pd.MultiIndex.from_tuples(all_cols)
all_err_metrics = pd.concat([all_err_metrics,  pd.DataFrame({apppend_cols[i]: err_metrics_c_1.iloc[:,-3:].to_numpy()[:,i] for i in range(3)})], axis=1)

# get training error metrics
all_rec_err_metrics = rec_err_metrics_c_0.copy()
all_cols = all_rec_err_metrics.columns.to_list()
apppend_cols = []
for i in range(len(all_cols)):
    if all_cols[i][0] == 'MC':
        t = all_cols[i][1]
        all_cols[i] = ('MC - No Cluster', t)
        apppend_cols += [('MC - With Cluster', t)]
all_rec_err_metrics.columns = pd.MultiIndex.from_tuples(all_cols)
all_rec_err_metrics = pd.concat([all_rec_err_metrics,  pd.DataFrame({apppend_cols[i]: rec_err_metrics_c_1.iloc[:,-2:].to_numpy()[:,i] for i in range(2)})], axis=1)

# save to excel file
write_path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    write_path += f'_{func}'
write_path += '/Errors_Metrics.xlsx'
with pd.ExcelWriter(write_path) as writer:
    all_err_metrics.to_excel(writer, sheet_name='Testing')

    all_rec_err_metrics.to_excel(writer, sheet_name='Training')

In [45]:
## save sparsity matrix and visualization wrt to spartsity to excel
idx1_test = np.sum((err_metrics_c_0['IUPAC', 'Component 1'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)
idx2_test = np.sum((err_metrics_c_0['IUPAC', 'Component 2'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)

idx1_train = np.sum((rec_err_metrics_c_0['IUPAC', 'Component 1'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)
idx2_train = np.sum((rec_err_metrics_c_0['IUPAC', 'Component 2'][:-1].to_numpy().astype(str)[:, np.newaxis] == post_obj.c_all[np.newaxis,:])*np.arange(post_obj.c_all.shape[0]), axis=1)

# name of sheets
sheet_names = ['Sparsity Marix', 'MAE - Test (UNI)', 'RMSE - Test (UNI)', 'MARE - Test (UNI)', 'MAE - Train (UNI)', 'RMSE - Train(UNI)', 'MAE - Test (c=0)', 'RMSE - Test (c=0)', 'MARE - Test (c=0)', 'MAE - Train (c=0)', 'RMSE - Train (c=0)', 'MAE - Test (c=1)', 'RMSE - Test (c=1)', 'MARE - Test (c=1)', 'MAE - Train (c=1)', 'RMSE - Train (c=1)']

# create list of data frames
all_dfs = []

all_dfs += [np.zeros((post_obj.c_all.shape[0], post_obj.c_all.shape[0])).astype(int)]
all_dfs[0][idx1_train, idx2_train] = 1
all_dfs[0] = pd.DataFrame(all_dfs[0], index=post_obj.c_all, columns=post_obj.c_all)

# name of sheets
list_to_parse = ['' for j in range(len(sheet_names))]

for i in range(1, len(sheet_names)):
    sheet_name = sheet_names[i]
    # assign corresponding UNIFAC
    if "UNI" in sheet_name:
        if "Test" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['UNIFAC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['UNIFAC', 'RMSE'][:-1].to_numpy()
            elif "MARE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['UNIFAC', 'MARE'][:-1].to_numpy()
        elif "Train" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_0['UNIFAC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_0['UNIFAC', 'RMSE'][:-1].to_numpy()
    # assign corresponding c=0
    elif "c=0" in sheet_name:
        if "Test" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['MC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['MC', 'RMSE'][:-1].to_numpy()
            elif "MARE" in sheet_name:
                list_to_parse[i] = err_metrics_c_0['MC', 'MARE'][:-1].to_numpy()
        elif "Train" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_0['MC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_0['MC', 'RMSE'][:-1].to_numpy()
    # assign corresponding c=1
    elif "c=1" in sheet_name:
        if "Test" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = err_metrics_c_1['MC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = err_metrics_c_1['MC', 'RMSE'][:-1].to_numpy()
            elif "MARE" in sheet_name:
                list_to_parse[i] = err_metrics_c_1['MC', 'MARE'][:-1].to_numpy()
        elif "Train" in sheet_name:
            if "MAE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_1['MC', 'MAE'][:-1].to_numpy()
            elif "RMSE" in sheet_name:
                list_to_parse[i] = rec_err_metrics_c_1['MC', 'RMSE'][:-1].to_numpy()

    all_dfs += [np.nan*np.zeros((post_obj.c_all.shape[0], post_obj.c_all.shape[0]))]
    if "Train" in sheet_name:
        all_dfs[i][idx1_train, idx2_train] = list_to_parse[i]
    elif "Test" in sheet_name:
        all_dfs[i][idx1_test, idx2_test] = list_to_parse[i]
    all_dfs[i] = pd.DataFrame(all_dfs[i], index=post_obj.c_all, columns=post_obj.c_all)
    all_dfs[i][all_dfs[i].isna()] = ''

# save to excel file
write_path = f'{post_obj.init_path}/{functional_groups[0]}'
for func in functional_groups[1:]:
    write_path += f'_{func}'
write_path += '/Sparsity_Metrics.xlsx'

with pd.ExcelWriter(write_path) as writer:
    for i in range(len(sheet_names)):
        all_dfs[i].to_excel(writer, sheet_name=sheet_names[i], index=True, header=True)

/tmp/ipykernel_39440/214037210.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_dfs[i][all_dfs[i].isna()] = ''
/tmp/ipykernel_39440/214037210.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_dfs[i][all_dfs[i].isna()] = ''
/tmp/ipykernel_39440/214037210.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  all_dfs[i][all_dfs[i].isna()] = ''
/tmp/ipykernel_39440/214037210.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will r

In [46]:
write_path

'/home/garren/Documents/MEng/Code/Latest_results/HPC Files/Hybrid PMF/Subsets/all/Sparsity_Metrics.xlsx'